# Catboost ensemble_starter

In [1]:
#load the package
import numpy as np 
import pandas as pd 
from catboost import CatBoostRegressor
from tqdm import tqdm
import datetime as dt

# Load the data, formatting data

In [2]:
#load the properties data
properties2016 = pd.read_csv('properties_2016.csv', low_memory = False)
properties2017 = pd.read_csv('properties_2017.csv', low_memory = False)
#load the train data
train2016 = pd.read_csv('train_2016_v2.csv', parse_dates=['transactiondate'], low_memory=False)
train2017 = pd.read_csv('train_2017.csv', parse_dates=['transactiondate'], low_memory=False)
#load sample data
sample_submission = pd.read_csv('sample_submission.csv', low_memory = False)

In [3]:
def trans_datetime_features(df):
    df["year"] = df["transactiondate"].dt.year
    df["quarter"] = (df["transactiondate"].dt.year - 2016)*4 +df["transactiondate"].dt.quarter
    df["month"] = (df["transactiondate"].dt.year - 2016)*12 + df["transactiondate"].dt.month
    df["day"] = df["transactiondate"].dt.day
    df.drop(["transactiondate"], inplace=True, axis=1)
    return df

In [4]:
# transform datatime feature
train2016 = trans_datetime_features(train2016)
train2017 = trans_datetime_features(train2017)
#merge the properties and train data 
train2016 = pd.merge(train2016, properties2016, how = 'left', on = 'parcelid')
train2017 = pd.merge(train2017, properties2017, how = 'left', on = 'parcelid')

In [5]:
#tax feature in 2017, np.nan
train2017.iloc[:, train2017.columns.str.startswith('tax')] = np.nan
#concat Train 2016, 2017
train_df = pd.concat([train2016, train2017], axis = 0)
#merge submission, properties2016
test_df = pd.merge(sample_submission[['ParcelId']], properties2016.rename(columns = {'parcelid': 'ParcelId'}), how = 'left', on = 'ParcelId')

In [6]:
train_df.shape, test_df.shape

((167888, 63), (2985217, 58))

# Exclue Missing data > 98%

In [7]:
# find the columns with the missing values, and set the threshhold 98
missing = pd.concat([train_df.isnull().sum(), 100 * train_df.isnull().mean()], axis=1)
missing.columns=['count', '%']
remove_missing_cols = missing[missing.iloc[:,1] > 98].index.tolist()
print("Remove containing missing values 98 percent of cols: %s" % len(remove_missing_cols))
print('Remove_missing_cols are: \n %s' % remove_missing_cols)

Remove containing missing values 98 percent of cols: 15
Remove_missing_cols are: 
 ['architecturalstyletypeid', 'basementsqft', 'buildingclasstypeid', 'decktypeid', 'finishedsquarefeet13', 'finishedsquarefeet6', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'storytypeid', 'typeconstructiontypeid', 'yardbuildingsqft26', 'fireplaceflag', 'taxdelinquencyflag', 'taxdelinquencyyear']


# Exclue features that only have one values

In [8]:
# check the cols that with one unique values, and remove them

remove_unique_col = []
for col in train_df.columns:
    num_of_uniques = len(train_df[col].unique())
    if train_df[col].isnull().sum() != 0:
        num_of_uniques -= 1
    if num_of_uniques == 1:
        remove_unique_col.append(col)

print("Remove the cols with unique value: %s" % len(remove_unique_col))
print('Removed unique value cols are: \n %s' % remove_unique_col)

Remove the cols with unique value: 9
Removed unique value cols are: 
 ['decktypeid', 'hashottuborspa', 'poolcnt', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'storytypeid', 'fireplaceflag', 'taxdelinquencyflag']


# Prepare training features

In [11]:
exclude_list = ['parcelid', 'propertyzoningdesc', 'logerror', 'propertycountylandusecode']
exclude_list = exclude_list + remove_missing_cols + remove_unique_col
train_features = [e for e in train_df.columns if e not in exclude_list]
print("Training features are: %s" % len(train_features))
print('Training features are: \n %s ' % train_features)

Training features are: 41
Training features are: 
 ['year', 'quarter', 'month', 'day', 'airconditioningtypeid', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr', 'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'finishedsquarefeet15', 'finishedsquarefeet50', 'fips', 'fireplacecnt', 'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet', 'propertylandusetypeid', 'rawcensustractandblock', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt', 'threequarterbathnbr', 'unitcnt', 'yardbuildingsqft17', 'yearbuilt', 'numberofstories', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt', 'taxamount', 'censustractandblock'] 


# Define categorical features among training features

In [12]:
feature_nunique = pd.DataFrame(train_df[train_features].nunique(), columns=['count'])
cat_possible_list = feature_nunique[feature_nunique['count'] < 1000].index.tolist()
cat_feature_list = [f for f in cat_possible_list if 'sqft' not in f and 'cnt' not in f and 'nbr' not in f and 'number' not in f]
cat_features = [train_df[train_features].columns.get_loc(col) for col in cat_feature_list]
print ("Number of categorical features are defined while they have less than 1000 unique values and \n have no numeric types of features in the list: %s" % len(cat_feature_list))
print('\n', '************************************************************************************************************')
print ("Categorical features are defined while they have less than 1000 unique values and \n have no numeric types of features in the list: \n %s" % cat_feature_list)


Number of categorical features are defined while they have less than 1000 unique values and 
 have no numeric types of features in the list: 15

 ************************************************************************************************************
Categorical features are defined while they have less than 1000 unique values and 
 have no numeric types of features in the list: 
 ['year', 'quarter', 'month', 'day', 'airconditioningtypeid', 'buildingqualitytypeid', 'fips', 'heatingorsystemtypeid', 'propertylandusetypeid', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'yearbuilt', 'assessmentyear']


# Fill NA, and TRAINING

In [17]:
# fillna with 'None' string before calculating hashes.
# helpfullink:https://github.com/catboost/catboost/blob/master/open_problems/open_problems.md

train_df.replace(np.nan, 'None', inplace=True)
test_df.replace(np.nan, 'None', inplace=True)


In [19]:

# separate predictor variables and target variable 
X_train = train_df[train_features]
y_train = train_df.logerror
print("X_train.shape, y_train.shape are: ", X_train.shape, y_train.shape)

# creat transactiondate column and assign timestamp
test_df['transactiondate'] = pd.Timestamp('2016-12-01') 
test_df = trans_datetime_features(test_df)
X_test = test_df[train_features]
print("X_test.shape is: ", X_test.shape)

X_train.shape, y_train.shape are:  (167888, 41) (167888,)
X_test.shape is:  (2985217, 41)


In [24]:
X_train[cat_feature_list] = X_train[cat_feature_list].astype(str)
X_test[cat_feature_list] = X_test[cat_feature_list].astype(str)

C:\Users\dongw\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [25]:
num_ensembles = 5
y_pred = 0.0
for i in tqdm(range(num_ensembles)):
    model = CatBoostRegressor(
        iterations=630, learning_rate=0.03,
        depth=6, l2_leaf_reg=3,
        loss_function='MAE',
        eval_metric='MAE',
        random_seed=i)
    model.fit(
        X_train, y_train,
        cat_features=cat_features)
    y_pred += model.predict(X_test)
y_pred /= num_ensembles


  0%|          | 0/5 [00:00<?, ?it/s]

0:	learn: 0.0688436	total: 341ms	remaining: 3m 34s
1:	learn: 0.0688280	total: 513ms	remaining: 2m 40s
2:	learn: 0.0688137	total: 680ms	remaining: 2m 22s
3:	learn: 0.0687987	total: 832ms	remaining: 2m 10s
4:	learn: 0.0687850	total: 997ms	remaining: 2m 4s
5:	learn: 0.0687704	total: 1.22s	remaining: 2m 6s
6:	learn: 0.0687583	total: 1.4s	remaining: 2m 4s
7:	learn: 0.0687450	total: 1.58s	remaining: 2m 3s
8:	learn: 0.0687324	total: 1.76s	remaining: 2m 1s
9:	learn: 0.0687211	total: 1.91s	remaining: 1m 58s
10:	learn: 0.0687105	total: 2.08s	remaining: 1m 56s
11:	learn: 0.0687001	total: 2.29s	remaining: 1m 57s
12:	learn: 0.0686910	total: 2.42s	remaining: 1m 55s
13:	learn: 0.0686813	total: 2.59s	remaining: 1m 53s
14:	learn: 0.0686723	total: 2.74s	remaining: 1m 52s
15:	learn: 0.0686631	total: 2.92s	remaining: 1m 51s
16:	learn: 0.0686533	total: 3.09s	remaining: 1m 51s
17:	learn: 0.0686447	total: 3.31s	remaining: 1m 52s
18:	learn: 0.0686371	total: 3.46s	remaining: 1m 51s
19:	learn: 0.0686304	total: 

159:	learn: 0.0681933	total: 29.3s	remaining: 1m 26s
160:	learn: 0.0681919	total: 29.6s	remaining: 1m 26s
161:	learn: 0.0681910	total: 29.8s	remaining: 1m 25s
162:	learn: 0.0681891	total: 29.9s	remaining: 1m 25s
163:	learn: 0.0681878	total: 30.1s	remaining: 1m 25s
164:	learn: 0.0681867	total: 30.2s	remaining: 1m 25s
165:	learn: 0.0681860	total: 30.3s	remaining: 1m 24s
166:	learn: 0.0681849	total: 30.5s	remaining: 1m 24s
167:	learn: 0.0681822	total: 30.7s	remaining: 1m 24s
168:	learn: 0.0681815	total: 30.8s	remaining: 1m 24s
169:	learn: 0.0681794	total: 31s	remaining: 1m 23s
170:	learn: 0.0681780	total: 31.1s	remaining: 1m 23s
171:	learn: 0.0681760	total: 31.3s	remaining: 1m 23s
172:	learn: 0.0681753	total: 31.4s	remaining: 1m 23s
173:	learn: 0.0681736	total: 31.6s	remaining: 1m 22s
174:	learn: 0.0681716	total: 31.7s	remaining: 1m 22s
175:	learn: 0.0681701	total: 31.9s	remaining: 1m 22s
176:	learn: 0.0681686	total: 32s	remaining: 1m 21s
177:	learn: 0.0681675	total: 32.2s	remaining: 1m 2

317:	learn: 0.0680195	total: 57s	remaining: 55.9s
318:	learn: 0.0680180	total: 57.1s	remaining: 55.7s
319:	learn: 0.0680170	total: 57.3s	remaining: 55.5s
320:	learn: 0.0680152	total: 57.5s	remaining: 55.3s
321:	learn: 0.0680147	total: 57.6s	remaining: 55.1s
322:	learn: 0.0680134	total: 57.8s	remaining: 54.9s
323:	learn: 0.0680117	total: 57.9s	remaining: 54.7s
324:	learn: 0.0680112	total: 58.1s	remaining: 54.6s
325:	learn: 0.0680100	total: 58.3s	remaining: 54.4s
326:	learn: 0.0680089	total: 58.4s	remaining: 54.1s
327:	learn: 0.0680081	total: 58.6s	remaining: 54s
328:	learn: 0.0680074	total: 58.8s	remaining: 53.8s
329:	learn: 0.0680066	total: 59s	remaining: 53.6s
330:	learn: 0.0680050	total: 59.1s	remaining: 53.4s
331:	learn: 0.0680042	total: 59.3s	remaining: 53.2s
332:	learn: 0.0680038	total: 59.5s	remaining: 53.1s
333:	learn: 0.0680034	total: 59.7s	remaining: 52.9s
334:	learn: 0.0680025	total: 59.9s	remaining: 52.7s
335:	learn: 0.0680006	total: 1m	remaining: 52.5s
336:	learn: 0.0680005

475:	learn: 0.0678576	total: 1m 25s	remaining: 27.6s
476:	learn: 0.0678572	total: 1m 25s	remaining: 27.5s
477:	learn: 0.0678562	total: 1m 25s	remaining: 27.3s
478:	learn: 0.0678542	total: 1m 25s	remaining: 27.1s
479:	learn: 0.0678534	total: 1m 26s	remaining: 26.9s
480:	learn: 0.0678518	total: 1m 26s	remaining: 26.7s
481:	learn: 0.0678513	total: 1m 26s	remaining: 26.6s
482:	learn: 0.0678506	total: 1m 26s	remaining: 26.4s
483:	learn: 0.0678482	total: 1m 26s	remaining: 26.2s
484:	learn: 0.0678457	total: 1m 27s	remaining: 26s
485:	learn: 0.0678437	total: 1m 27s	remaining: 25.8s
486:	learn: 0.0678427	total: 1m 27s	remaining: 25.7s
487:	learn: 0.0678412	total: 1m 27s	remaining: 25.5s
488:	learn: 0.0678401	total: 1m 27s	remaining: 25.3s
489:	learn: 0.0678390	total: 1m 28s	remaining: 25.1s
490:	learn: 0.0678387	total: 1m 28s	remaining: 25s
491:	learn: 0.0678377	total: 1m 28s	remaining: 24.8s
492:	learn: 0.0678366	total: 1m 28s	remaining: 24.6s
493:	learn: 0.0678347	total: 1m 28s	remaining: 24.

...

158:	learn: 0.0681945	total: 28.9s	remaining: 1m 25s
159:	learn: 0.0681933	total: 29.2s	remaining: 1m 25s
160:	learn: 0.0681926	total: 29.3s	remaining: 1m 25s
161:	learn: 0.0681907	total: 29.6s	remaining: 1m 25s
162:	learn: 0.0681889	total: 29.8s	remaining: 1m 25s
163:	learn: 0.0681878	total: 29.9s	remaining: 1m 24s
164:	learn: 0.0681856	total: 30.2s	remaining: 1m 24s
165:	learn: 0.0681842	total: 30.4s	remaining: 1m 24s
166:	learn: 0.0681822	total: 30.6s	remaining: 1m 24s
167:	learn: 0.0681810	total: 30.8s	remaining: 1m 24s
168:	learn: 0.0681795	total: 31s	remaining: 1m 24s
169:	learn: 0.0681790	total: 31.2s	remaining: 1m 24s
170:	learn: 0.0681767	total: 31.3s	remaining: 1m 24s
171:	learn: 0.0681756	total: 31.6s	remaining: 1m 24s
172:	learn: 0.0681736	total: 31.8s	remaining: 1m 23s
173:	learn: 0.0681728	total: 32s	remaining: 1m 23s
174:	learn: 0.0681719	total: 32.2s	remaining: 1m 23s
175:	learn: 0.0681706	total: 32.4s	remaining: 1m 23s
176:	learn: 0.0681696	total: 32.6s	remaining: 1m 2

315:	learn: 0.0680267	total: 59.2s	remaining: 58.8s
316:	learn: 0.0680258	total: 59.3s	remaining: 58.6s
317:	learn: 0.0680249	total: 59.5s	remaining: 58.4s
318:	learn: 0.0680238	total: 59.7s	remaining: 58.2s
319:	learn: 0.0680225	total: 59.9s	remaining: 58s
320:	learn: 0.0680211	total: 1m	remaining: 57.8s
321:	learn: 0.0680208	total: 1m	remaining: 57.6s
322:	learn: 0.0680195	total: 1m	remaining: 57.4s
323:	learn: 0.0680187	total: 1m	remaining: 57.2s
324:	learn: 0.0680180	total: 1m	remaining: 57s
325:	learn: 0.0680172	total: 1m	remaining: 56.8s
326:	learn: 0.0680158	total: 1m 1s	remaining: 56.6s
327:	learn: 0.0680149	total: 1m 1s	remaining: 56.4s
328:	learn: 0.0680144	total: 1m 1s	remaining: 56.2s
329:	learn: 0.0680135	total: 1m 1s	remaining: 56s
330:	learn: 0.0680127	total: 1m 1s	remaining: 55.8s
331:	learn: 0.0680121	total: 1m 1s	remaining: 55.6s
332:	learn: 0.0680117	total: 1m 2s	remaining: 55.4s
333:	learn: 0.0680101	total: 1m 2s	remaining: 55.3s
334:	learn: 0.0680093	total: 1m 2s	r

472:	learn: 0.0678615	total: 1m 27s	remaining: 28.9s
473:	learn: 0.0678593	total: 1m 27s	remaining: 28.7s
474:	learn: 0.0678568	total: 1m 27s	remaining: 28.5s
475:	learn: 0.0678551	total: 1m 27s	remaining: 28.3s
476:	learn: 0.0678543	total: 1m 27s	remaining: 28.1s
477:	learn: 0.0678531	total: 1m 27s	remaining: 28s
478:	learn: 0.0678520	total: 1m 28s	remaining: 27.8s
479:	learn: 0.0678507	total: 1m 28s	remaining: 27.6s
480:	learn: 0.0678498	total: 1m 28s	remaining: 27.4s
481:	learn: 0.0678484	total: 1m 28s	remaining: 27.2s
482:	learn: 0.0678476	total: 1m 28s	remaining: 27s
483:	learn: 0.0678461	total: 1m 29s	remaining: 26.9s
484:	learn: 0.0678455	total: 1m 29s	remaining: 26.7s
485:	learn: 0.0678441	total: 1m 29s	remaining: 26.5s
486:	learn: 0.0678427	total: 1m 29s	remaining: 26.3s
487:	learn: 0.0678418	total: 1m 29s	remaining: 26.2s
488:	learn: 0.0678397	total: 1m 30s	remaining: 26s
489:	learn: 0.0678380	total: 1m 30s	remaining: 25.8s
490:	learn: 0.0678363	total: 1m 30s	remaining: 25.6s

628:	learn: 0.0676943	total: 1m 54s	remaining: 182ms
629:	learn: 0.0676927	total: 1m 54s	remaining: 0us


100%|██████████| 5/5 [10:58<00:00, 131.60s/it]


In [26]:
submission = pd.DataFrame({
    'ParcelId': test_df['ParcelId'],
})

test_dates = {
    '201610': pd.Timestamp('2016-09-30'),
    '201611': pd.Timestamp('2016-10-31'),
    '201612': pd.Timestamp('2016-11-30'),
    '201710': pd.Timestamp('2017-09-30'),
    '201711': pd.Timestamp('2017-10-31'),
    '201712': pd.Timestamp('2017-11-30')
}

for label, test_date in test_dates.items():
    print("Predicting for: %s . " % (label))
    submission[label] = y_pred
    
submission.to_csv('CatBoost_prediction.csv', float_format='%.6f',index=False)

Predicting for: 201610 . 
Predicting for: 201611 . 
Predicting for: 201612 . 
Predicting for: 201710 . 
Predicting for: 201711 . 
Predicting for: 201712 . 


In [27]:
submission

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0.054393,0.054393,0.054393,0.054393,0.054393,0.054393
1,10759547,0.050820,0.050820,0.050820,0.050820,0.050820,0.050820
2,10843547,-0.025382,-0.025382,-0.025382,-0.025382,-0.025382,-0.025382
3,10859147,0.027895,0.027895,0.027895,0.027895,0.027895,0.027895
4,10879947,0.007612,0.007612,0.007612,0.007612,0.007612,0.007612
...,...,...,...,...,...,...,...
2985212,168176230,0.010926,0.010926,0.010926,0.010926,0.010926,0.010926
2985213,14273630,0.010926,0.010926,0.010926,0.010926,0.010926,0.010926
2985214,168040630,0.010926,0.010926,0.010926,0.010926,0.010926,0.010926
2985215,168040830,0.010926,0.010926,0.010926,0.010926,0.010926,0.010926
